**TensorFlow多线程输入数据处理框架**  

---
当数据量很大时，入队操作从硬盘中读取数据，放入内存中，主线程需要等待入队操作完成，才能进行训练。会话中可以运行多个线程，实现异步读取。

In [1]:
import tensorflow as tf
import os

# 调整警告等级
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

## 队列
----

### 先进先出队列

In [2]:
with tf.Graph().as_default():
    # 模拟一下同步先处理数据，然后才能取数据训练
    # 1.首先定义一个队列
    queue1 = tf.FIFOQueue(capacity=3, dtypes=tf.float32)

    # 初始化，放入一些数据
    init = queue1.enqueue_many([[0.1, 0.2, 0.3], ])

    out_queue = queue1.dequeue()
    data = out_queue + 1
    en_queue = queue1.enqueue(data)

    with tf.Session() as sess:
        # 初始化队列
        sess.run(init)

        for i in range(3):
            sess.run(en_queue)

        # print(queue1.size().eval())

        for i in range(queue1.size().eval()):
            print(sess.run(out_queue))

1.1
1.2
1.3


### 随机队列
RandomShuffleQueue会 将队列中的元素打乱，每次出队列操作得到的是当前队列所有元素中随机选择的一个。

In [4]:
with tf.Graph().as_default():
    queue = tf.RandomShuffleQueue(capacity=5, min_after_dequeue=0, dtypes=tf.float32)

    # 放入一些数据
    init = queue.enqueue_many([[0.1, 0.2, 0.3, 0.4, 0.5], ])

    out_queue = queue.dequeue()

    with tf.Session() as sess:
        for i in range(5):
            sess.run(init)
            print('第{}次实验'.format(i+1))

            for m in range(queue.size().eval()):
                x = sess.run(out_queue)
                print('  第{}次弹出队列：{:.1f}'.format(m+1, x))

第1次实验
  第1次弹出队列：0.4
  第2次弹出队列：0.5
  第3次弹出队列：0.3
  第4次弹出队列：0.2
  第5次弹出队列：0.1
第2次实验
  第1次弹出队列：0.2
  第2次弹出队列：0.4
  第3次弹出队列：0.1
  第4次弹出队列：0.3
  第5次弹出队列：0.5
第3次实验
  第1次弹出队列：0.5
  第2次弹出队列：0.1
  第3次弹出队列：0.4
  第4次弹出队列：0.3
  第5次弹出队列：0.2
第4次实验
  第1次弹出队列：0.5
  第2次弹出队列：0.2
  第3次弹出队列：0.3
  第4次弹出队列：0.4
  第5次弹出队列：0.1
第5次实验
  第1次弹出队列：0.1
  第2次弹出队列：0.3
  第3次弹出队列：0.2
  第4次弹出队列：0.4
  第5次弹出队列：0.5


## 线程（队列管理器）

In [ ]:
with tf.Graph().as_default():
    queue = tf.FIFOQueue(1000, tf.float32)

    var = tf.Variable(0.0)

    # 实现自增
    data = tf.assign_add(var, tf.constant(1.0))

    en_q = queue.enqueue([data])

    # 定义队列管理器op，指定多少个子线程，以及子线程的任务
    qr = tf.train.QueueRunner(queue=queue, enqueue_ops=[en_q] * 2)

    # 初始化变量op
    init_op = tf.global_variables_initializer()

    with tf.Session() as sess:
        # 初始化变量
        sess.run(init_op)
        
        # 开启子线程
        threads = qr.create_threads(sess, start=True)
        
        # 主线程，不断的读取数据
        for i in range(300):
            print(sess.run(queue.enqueue()))

TensorFlow提供一种TFRecord来存储数据，TFRecord文件中的数据都是通过tf.train.Example Protocol Buffer的格式来存储。